# 3교시 데이터 타입

> 스파크에서 사용되는 데이터 타입에 대해 실습합니다

## 목차
* [1. 리터럴 타입](#1.-리터럴-타입)
* [2. 불리언 형 데이터 타입 다루기](#2.-불리언-형-데이터-타입-다루기)
* [3. 수치형 데이터 타입 다루기](#3.-수치형-데이터-타입-다루기)
* [4. 문자열 데이터 타입 다루기](#4.-문자열-데이터-타입-다루기)
* [5. 정규 표현식](#5.-정규-표현식)
* [6. 날짜와 타임스팸프 데이터 타입 다루기](#6.-날짜와-타임스팸프-데이터-타입-다루기)
* [7. 널 값 다루기](#7.-널-값-다루기)
* [참고자료](#참고자료)
 


In [68]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

In [69]:
""" DataFrame 생성 """
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/by-day/2010-12-01.csv")
)
df.printSchema()
df.createOrReplaceTempView("retail")
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   8

## 1. 리터럴 타입

In [3]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0)).limit(5)

5,five,5.0
5,five,5.0
5,five,5.0
5,five,5.0
5,five,5.0
5,five,5.0


## 2. 불리언 형 데이터 타입 다루기
### 2.1 AND 조건

In [4]:
from pyspark.sql.functions import col

x1 = df.where(col("InvoiceNO") != 536365).select("InvoiceNO", "Description")
x2 = df.where("InvoiceNO <> 536365").select("InvoiceNO", "Description")
x3 = df.where("InvoiceNO = 536365").select("InvoiceNO", "Description")

x1.show(2)
x2.show(2)

+---------+--------------------+
|InvoiceNO|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
+---------+--------------------+
only showing top 2 rows

+---------+--------------------+
|InvoiceNO|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
+---------+--------------------+
only showing top 2 rows



### 2.2 OR 조건

In [16]:
from pyspark.sql.functions import instr
# df.where( (col("UnitPrice") > 600) | (instr(col("Description"), "POSTAGE") >= 1) ).show()
df.where("UnitPrice > 600 OR instr(Description, 'POSTAGE') >= 1").show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536370|     POST|       POSTAGE|       3|2010-12-01 08:45:00|     18.0|   12583.0|        France|
|   536403|     POST|       POSTAGE|       1|2010-12-01 11:27:00|     15.0|   12791.0|   Netherlands|
|   536527|     POST|       POSTAGE|       1|2010-12-01 13:04:00|     18.0|   12662.0|       Germany|
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

+---------+---------+--------------+--------+-------------------+---------+------

### 2.3 ISIN - 제공된 목록에 포함되었는지 여부

In [18]:
# SparkSQL 을 이용한 is in 구문 사용
from pyspark.sql.functions import desc
# df.select("StockCode").where(col("StockCode").isin(["DOT", "POST", "C2"])).distinct().show()
df.select('StockCode').where("StockCode in ('DOT', 'POST', 'C2')").distinct().show()

+---------+
|StockCode|
+---------+
|     POST|
|       C2|
|      DOT|
+---------+

+---------+
|StockCode|
+---------+
|     POST|
|       C2|
|      DOT|
+---------+



### 2.4 INSTR - 특정 문자열이 포함되었는지 여부

In [7]:
from pyspark.sql.functions import *
""" instr 함수 """
df.withColumn("added", instr(df.Description, "POSTAGE")).where("added > 1").show() # 8번째 글자에 'POSTAGE'가 시작됨

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|added|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|    8|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|    8|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+



### <font color=green>1. [기본]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요
#### 3. 송장번호(InvoiceNo) 가 '536365' 이면서
#### 4. 상품코드(StockCode) 가 ('85123A', '84406B', '84029G', '84029E') 중에 하나이면서
#### 5. 제품단가(UnitPrice) 가 2.6 이하 혹은 3.0 이상인 경우를 출력하세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df1 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/by-day/2010-12-01.csv")
)
df1.printSchema()
df1.show(10)
answer = df1.where("InvoiceNo = '536365'").where("StockCode in ('85123A', '84406B', '84029G', '84029E')").where("UnitPrice < 2.6 or UnitPrice > 3.0")
answer.show()
```

</details>


In [8]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   8

### <font color=blue>2. [중급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 상품코드(StockCode) 가 ("POST", "M", "DOT", "D", "C2") 혹은 제품단가(UnitPrice) 가 30 이상인 데이터에 대하여
#### 2. 상품코드(StockCode) 기준의 빈도(count)는 얼마인지 출력하세요
* Structured API를 활용하여 작성하세요
* StockCode 내림 차순으로 정렬하세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df2 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
df2.where( (col("StockCode").isin(["POST", "M", "DOT", "D", "C2"])) | (col("UnitPrice") >= 30)).groupBy("StockCode").count().orderBy(desc("StockCode"))
```

</details>

In [33]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


StockCode,count
POST,3
M,2
DOT,2
D,1
C2,1
22947,1
22946,1
22847,1
22827,1
22803,2


## 3. 수치형 데이터 타입 다루기
### 3.1 각종 함수를 표현식으로 작성합니다

In [9]:
from pyspark.sql.functions import expr, pow
df.selectExpr("CustomerID", "pow(Quantity * UnitPrice, 2) + 5 as realQuantity").show(2)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [10]:
""" 지수만큼 제곱하는 pow 함수를 API를 사용해도 결과는 동일합니다 """
from pyspark.sql.functions import expr, pow

# 아래의 연산이 필요한 경우에는 반드시 column 으로 지정되어야 연산자 계산이 됩니다. (문자열 * 연산자는 없습니다)
fabricateQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), fabricateQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



### 3.2 반올림(round), 올림(ceil), 버림(floor)

In [11]:
from pyspark.sql.functions import *
df.selectExpr("round(2.5, 0)", "ceil(2.4)", "floor(2.6)").show(1)

+-------------+---------+----------+
|round(2.5, 0)|CEIL(2.4)|FLOOR(2.6)|
+-------------+---------+----------+
|            3|        3|         2|
+-------------+---------+----------+
only showing top 1 row



### 3.3 요약 통계

In [12]:
df.describe().show()
df.describe("InvoiceNo").show() # 컬럼을 입력

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

### <font color=green>3. [기본]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요
#### 3. 송장번호(InvoiceNo) 가 '536367' 인 거래 내역의
#### 4. 총 금액 (TotalPrice) = 수량(Quantity) * 단가(UnitPrice) 를 계산하여 TotalPrice 컬럼을 추가하세요
#### 5. 단, 총 금액 (TotalPrice) 계산시에 소수점 이하는 버림으로 처리하세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df3 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/by-day/2010-12-01.csv")
)
df3.printSchema()
df3.show(10)
answer = df3.where("InvoiceNo = '536367'").withColumn("TotalPrice", expr("floor(Quantity * UnitPrice)"))
display(answer)
```

</details>


In [9]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   8

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54
536367,22745,POPPY'S PLAYHOUSE BEDROOM,6,2010-12-01 08:34:00,2.1,13047.0,United Kingdom,12
536367,22748,POPPY'S PLAYHOUSE KITCHEN,6,2010-12-01 08:34:00,2.1,13047.0,United Kingdom,12
536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,2010-12-01 08:34:00,3.75,13047.0,United Kingdom,30
536367,22310,IVORY KNITTED MUG COSY,6,2010-12-01 08:34:00,1.65,13047.0,United Kingdom,9
536367,84969,BOX OF 6 ASSORTED COLOUR TEASPOONS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,25
536367,22623,BOX OF VINTAGE JIGSAW BLOCKS,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14
536367,22622,BOX OF VINTAGE ALPHABET BLOCKS,2,2010-12-01 08:34:00,9.95,13047.0,United Kingdom,19
536367,21754,HOME BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17
536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17


## 4. 문자열 데이터 타입 다루기
### 4.1 첫 문자열만 대문자로 변경
* 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경, initcap

In [14]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(2, False)

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|White Hanging Heart T-light Holder|
|White Metal Lantern               |
+----------------------------------+
only showing top 2 rows



### 4.2 대문자(upper), 소문자(lower)

In [15]:
from pyspark.sql.functions import lower, upper
df.selectExpr("Description", "lower(Description)", "upper(Description)").show(2)

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+
only showing top 2 rows



### 4.3 문자열 주변의 공백을 제거, lpad/ltrim/rpad/rtrim/trim

In [16]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
    ltrim(lit("   HELLO   ")).alias("ltrim"),
    rtrim(lit("   HELLO   ")).alias("rtrim"),
    trim(lit("   HELLO   ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")
).show(2)

+--------+--------+-----+---+----------+
|   ltrim|   rtrim| trim| lp|        rp|
+--------+--------+-----+---+----------+
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
+--------+--------+-----+---+----------+
only showing top 2 rows



### <font color=blue>4. [중급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 송장번호(InvoiceNo) 가 '536365' 인 거래 내역의 제품코드(StockCode) 를 총 8자리 문자로 출력해 주세요
* 제품코드의 출력 시의 빈 앞자리는 0으로 채워주세요 (Padding)
* 0이 패딩된 제품코드(StockCode) 컬럼의 컬럼명은 StockCode 로 유지되어야 합니다
* 최종 출력되는 컬럼은 "InvoiceNo", "StockCode", "Description" 만 출력하세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 보세요

<details><summary>[실습4] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df4 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
df4.where("InvoiceNo = '536365'").select("InvoiceNo", lpad("StockCode", 8, "0").alias("StockCode"), "Description")
```

</details>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


InvoiceNo,StockCode,Description
536365,0085123A,WHITE HANGING HEART T-LIGHT HOLDER
536365,00071053,WHITE METAL LANTERN
536365,0084406B,CREAM CUPID HEARTS COAT HANGER
536365,0084029G,KNITTED UNION FLAG HOT WATER BOTTLE
536365,0084029E,RED WOOLLY HOTTIE WHITE HEART.
536365,00022752,SET 7 BABUSHKA NESTING BOXES
536365,00021730,GLASS STAR FROSTED T-LIGHT HOLDER


<br>

## 5. 정규 표현식 & 조건부 컬럼

### 5.1 단어 치환, regexp_extract
* 존재 여부를 확인하거나 일치하는 모든 문자열을 치환
* 정규 표현식을 위해 regexp_extract 함수와 regexp_replace 함수를 제공


In [18]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GRENN|BLUE"
df.select(regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"), col("Description")).show(2, truncate=False)

+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [19]:
""" 문자 치환, translate """
from pyspark.sql.functions import translate
df.select(
    col("Description"),
    translate(col("Description"), "LEET", "12").alias("Translated") # 정확히 매칭되지 않아도 부분만 적용됩니다 L:1, E:2
).show(5)

+--------------------+--------------------+
|         Description|          Translated|
+--------------------+--------------------+
|WHITE HANGING HEA...|WHI2 HANGING H2AR...|
| WHITE METAL LANTERN|    WHI2 M2A1 1AN2RN|
|CREAM CUPID HEART...|CR2AM CUPID H2ARS...|
|KNITTED UNION FLA...|KNI2D UNION F1AG ...|
|RED WOOLLY HOTTIE...|R2D WOO11Y HOI2 W...|
+--------------------+--------------------+
only showing top 5 rows



In [20]:
""" 단어 추출, regexp_extract """
from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GRENN|BLUE)"
df.select(
    col("Description"),
    regexp_extract(col("Description"), extract_str, 1).alias("Extracted")
).show(2)

+--------------------+---------+
|         Description|Extracted|
+--------------------+---------+
|WHITE HANGING HEA...|    WHITE|
| WHITE METAL LANTERN|    WHITE|
+--------------------+---------+
only showing top 2 rows



In [21]:
""" 단어 존재유무, contain """ # 파이썬과 SQL은 instr 함수를 사용
from pyspark.sql.functions import instr

containBlack = instr(col("Description"), "BLACK") > 1
containWhite = instr(col("Description"), "WHITE") > 1
df.withColumn("hasSimpleColor", containBlack | containWhite) \
    .where("hasSimpleColor") \
    .select("Description") \
    .show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|RED WOOLLY HOTTIE WHITE HEART.    |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|WOOD S/3 CABINET ANT WHITE FINISH |
+----------------------------------+
only showing top 3 rows



In [22]:
""" 필드에 색깔 문자열이 포함되어 있는지 여부를 locate 함수를 이용하여 컬럼으로 생성하는 에제 """
from pyspark.sql.functions import expr, locate 

simple_colors = ["black", "white", "red", "green", "blue"]

def color_locator(column, color_string):   # color_strings 단어가 시작되는 문자기준(단어기준 X) 위치
    return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string)

selected_cols = [color_locator(df.Description, c) for c in simple_colors] # locate 함수를 하나씩 list에 저장
selected_cols.append(expr("*")) # column 타입이여야 함 

# * means -> Unnest List to variable arguments
df.select(*selected_cols).show(3)

df.select(*selected_cols).where(expr("is_white OR is_red")) \
    .select(col("Description")) \
    .show(3, False)

+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   false|    true| false|   false|  false|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   false|    true| false|   false|  false|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   false|   false| false|   false|  false|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------

### 5.2 조건부 컬럼 생성, when case else

* IF ELSE 와 같이 조건에 부합하는 경우에 따라 컬럼을 반환
* 조건을 중첩시켜서 N개의 조건을 적용할 수 있습니다

In [75]:
(
    df.withColumn(
        "Size", 
        when(col("Quantity") > 500, "Large")
        .when(col("Quantity") > 250, "Middle")
        .otherwise("Small")
    )
    .select("Quantity", "size")
    .orderBy(desc("Quantity"))
).show(10)

spark.sql("""
    select Quantity,
        case when Quantity > 500 then 'large' 
        when Quantity > 250 then 'middle' 
        else 'small' end as size 
    from retail 
    order by Quantity desc
""").show(10)

+--------+------+
|Quantity|  size|
+--------+------+
|     600| Large|
|     480|Middle|
|     432|Middle|
|     432|Middle|
|     384|Middle|
|     288|Middle|
|     252|Middle|
|     216| Small|
|     200| Small|
|     200| Small|
+--------+------+
only showing top 10 rows

+--------+------+
|Quantity|  size|
+--------+------+
|     600| large|
|     480|middle|
|     432|middle|
|     432|middle|
|     384|middle|
|     288|middle|
|     252|middle|
|     216| small|
|     200| small|
|     200| small|
+--------+------+
only showing top 10 rows



### 5-3. 사용자 정의 함수 

> User defined function(UDF)는 레포트별로 데이터를 처리하는 함수이며, SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록됨

* 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하 발생
* 언어별로 성능차이가 존재, 파이썬에서도 사용할 수 있으므로 자바나 스칼라도 함수 작성을 추천

In [122]:
""" UDF 사용하기 """
udfExDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

In [131]:
""" UDF 등록 및 사용 """
from pyspark.sql.functions import udf
power3udf = udf(power3)
udfExDF.select(power3udf(col("num"))).show()

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



### <font color=blue>5. [중급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 송장번호(InvoiceNo) 가 '536365' 인 거래 내역의 제품코드(StockCode) 를 총 8자리 문자로 출력해 주세요
* 제품코드의 출력 시의 빈 앞자리는 0으로 채워주세요 (Padding)
* 0이 패딩된 제품코드(StockCode) 컬럼의 컬럼명은 StockCode 로 유지되어야 합니다
* 최종 출력되는 컬럼은 "InvoiceNo", "StockCode", "Description" 만 출력하세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 보세요

<details><summary>[실습5] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df5 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
df5.where("InvoiceNo = '536365'").select("InvoiceNo", lpad("StockCode", 8, "0").alias("StockCode"), "Description")
```

</details>


In [77]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


InvoiceNo,StockCode,Description
536365,0085123A,WHITE HANGING HEART T-LIGHT HOLDER
536365,00071053,WHITE METAL LANTERN
536365,0084406B,CREAM CUPID HEARTS COAT HANGER
536365,0084029G,KNITTED UNION FLAG HOT WATER BOTTLE
536365,0084029E,RED WOOLLY HOTTIE WHITE HEART.
536365,00022752,SET 7 BABUSHKA NESTING BOXES
536365,00021730,GLASS STAR FROSTED T-LIGHT HOLDER


### <font color=red>6. [고급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. DESCRIPTION 항목에서 'GREEN' -> '초급', 'BLUE' -> '중급', 'RED' -> '고급' 으로 변경하여 출력하세요
* 해당 함수를 colorGrade(column) 라는 함수를 이용해서 작성해 보세요
* 최종 출력 시에는 Description 컬럼에 '급' 문자를 포함한 결과만 출력하세요

<details><summary>[실습6] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df6 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
df6.printSchema()
df6.show(3)

def colorReplacer(column):
    return (
        when(column.contains("GREEN"), regexp_replace(column, "GREEN", "초급"))
            .when(column.contains("BLUE"), regexp_replace(column, "BLUE", "중급"))
            .when(column.contains("RED"), regexp_replace(column, "RED", "고급"))
            .otherwise(column).alias("Description")
    )
    
df6.select(colorReplacer(col("Description"))).where(col("Description").contains("급")).show(truncate=False)
```

</details>


In [146]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+---------+----

## 6. 날짜와 타임스팸프 데이터 타입 다루기
> 시간대 설정이 필요하다면 스파크 SQL 설정의 spark.conf.sessionLocalTimeZone 속성으로 가능 <br>
> TimestampType 클래스는 초 단위 정밀도만 지원 - 초 단위 이상 정밀도 요구 시 long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책이 필요 <br>

* 스파크는 2가지 시간 정보만 다룸
  - 날짜 정보만 가지는 date
  - 날짜와 시간 정보를 모두 가지는 timestamp
* 시간대 설정이 필요하다면 스파크 SQL 설정의 spark.conf.sessionLocalTimeZone 속성으로 가능
  - 자바 TimeZone 포맷을 따라야 함
* TimestampType 클래스는 초 단위 정밀도만 지원
  - 초 단위 이상 정밀도 요구 시 long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책이 필요

### 6.1 오늘 날짜 구하기

In [23]:
from pyspark.sql.functions import current_date, current_timestamp

dateDF = (
    spark.range(10)
    .withColumn("today", current_date())
    .withColumn("now", current_timestamp())
)

dateDF.createOrReplaceTempView("dataTable")
dateDF.printSchema()

dateDF.show(3, False)

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2021-08-21|2021-08-21 17:49:05.713|
|1  |2021-08-21|2021-08-21 17:49:05.713|
|2  |2021-08-21|2021-08-21 17:49:05.713|
+---+----------+-----------------------+
only showing top 3 rows



### 6.2 날짜를 더하거나 빼기

In [24]:
from pyspark.sql.functions import date_sub, date_add
dateDF.select(
    date_sub(col("today"), 5),
    date_add(col("today"), 5)
).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-08-16|        2021-08-26|
+------------------+------------------+
only showing top 1 row



In [25]:
""" 두 날짜 사이의 일/개월 수를 파악 """
from pyspark.sql.functions import datediff, months_between, to_date

(
    dateDF.withColumn("week_ago", date_sub(col("today"), 7))
    .select(datediff(col("week_ago"), col("today")))
).show(1) # 현재 날짜에서 7일 제외 후 datediff 결과 확인

(
    dateDF
    .select(to_date(lit("2016-01-01")).alias("start"), to_date(lit("2017-05-22")).alias("end"))
    .select(months_between(col("start"), col("end")))
).show(1) # 개월 수 차이 파악

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [26]:
""" 문자열을 날짜로 변환 """ # 자바의 simpleDateFormat 클래스가 지원하는 포맷 사용 필요
from pyspark.sql.functions import to_date, lit

spark.range(5) \
    .withColumn("date", lit("2017-01-01")) \
    .select(to_date(col("date"))) \
    .show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



In [27]:
""" 파싱오류로 날짜가 null로 반환되는 사례 """
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1) # 월과 일의 순서가 바뀜

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



In [28]:
""" SimpleDateFormat 표준을 활용하여 날짜 포멧을 지정 """
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM" # 소문자 mm 주의
cleanDateDF = spark.range(1).select( # 1개 Row를 생성
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

※ SimpleDateFormat : https://bvc12.tistory.com/168

### 6.3 문자열을 날짜로 변환

In [29]:
from pyspark.sql.functions import to_date, lit

spark.range(5) \
    .withColumn("date", lit("2017-01-01")) \
    .select(to_date(col("date"))) \
    .show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



In [30]:
""" 파싱오류로 날짜가 null로 반환되는 사례 """
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1) # 월과 일의 순서가 바뀜

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



### <font color=blue>7. [중급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 적재일자(LoadDate) 컬럼을 넣되 포맷은 'yyyy-MM-dd' 으로 추가해 주시고 현재 일자를 넣으시면 됩니다
#### 2. 송장일자(InvoiceDate) 와 오늘 시간과의 차이를 나타내는 컬럼(InvoiceDiff)을 (LoadDate - to_date(InvoiceDate))넣어주세요
* 변경된 스키마를 출력하여 동일한 지 확인해 주세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 보세요

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df7 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
answer = df7.withColumn("LoadDate", current_date()).withColumn("InvoiceDiff", col("LoadDate") - to_date(col("InvoiceDate")))
display(answer)
answer.printSchema()
```

</details>


In [7]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+----------+-----------------+
|InvoiceNo|StockCode|                        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|  LoadDate|      InvoiceDiff|
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+----------+-----------------+
|   536365|   85123A| WHITE HANGING HEART T-LIGHT HOLDER|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|2021-09-01|10 years 9 months|
|   536365|    71053|                WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|2021-09-01|10 years 9 months|
|   536365|   84406B|     CREAM CUPID HEARTS COAT HANGER|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|2021-09-01|10 years 9 months|
|   536365|   84029G|KNITTED UNION FLAG HOT WATER BOTTLE|       6|2010-12-01 08:26:00|  

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- LoadDate: date (nullable = false)
 |-- InvoiceDiff: interval (nullable = true)



## 7. 널 값 다루기
+ null 값을 사용하는 것 보다 명시적으로 사용하는 것이 항상 좋음
+ null 값을 허용하지 않는 컬럼을 선언해도 강제성은 없음
+ nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 돕는 역할
+ null 값을 다루는 방법은 두 가지 
    + 명시적으로 null을 제거
    + 전역 또느 컬럼 단위로 null 값을 특정 값으로 채움

### 7-1. 컬럼 값에 따른 널 처리 함수 (ifnull, nullIf, nvl, nvl2)
+ SQL 함수이며 DataFrame의 select 표현식으로 사용 가능
    + ifnull(null, 'return_value') # 두 번째 값을, 아니라면 첫 번째 값을 반환 
    + nullif('value', 'value')     # 두 값이 같으면 null
    + nvl(null, 'return_value')    # 두 번째 값을, 아니라면 첫 번째 값을 반환
    + nvl2('not_null', 'return_value', 'else_value') # 두 번째 값을, 아니라면 세번째 값을 반환

In [32]:
spark.sql("""
SELECT
    ifnull(null, 'return_value'),
    nullif('value', 'value'),
    nvl(null, 'return_value'),
    nvl2('not null', 'return_value', 'else_value')
""").show()

+--------------------------+--------------------+-----------------------+----------------------------------------+
|ifnull(NULL, return_value)|nullif(value, value)|nvl(NULL, return_value)|nvl2(not null, return_value, else_value)|
+--------------------------+--------------------+-----------------------+----------------------------------------+
|              return_value|                null|           return_value|                            return_value|
+--------------------------+--------------------+-----------------------+----------------------------------------+



### 7-2 컬럼의 널 값에 따른 로우 제거 (na.drop)

In [33]:
df.na.drop()
df.na.drop("any").show(1) # 로우 컬럼값 중 하나라도 null이면 제거
df.na.drop("all").show(1) # 로우 컬럼값 모두 null이면 제거

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---

In [103]:
# 배열 형태의 컬럼을 인수로 전달하여 지정한 컬럼만 전체(all)가 null 인 경우만 제거합니다
df.na.drop("all", subset=("StockCode", "InvoiceNo")).show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



### 7.3 컬럼의 널 값에 따른 값을 채움 (na.fill)

In [105]:
""" null을 포함한 DataFrame 행성 """
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

myManualSchema = StructType([
    StructField("string_null", StringType(), True),
    StructField("string2_null", StringType(), True),
    StructField("number_null", DoubleType(), True)
])

myRows = []
myRows.append(Row("Hello", "World", float(5))) # string 컬럼에 null 포함
myRows.append(Row("Hello", None, float(5))) # string 컬럼에 null 포함
myRows.append(Row(None, "World", None))     # number 컬럼에 null 포함
myRows.append(Row(None, None, None))        # 모든 컬럼이 null

myDf = spark.createDataFrame(myRows, myManualSchema)
myDf.show()

myDf.na.fill( {"number_null": 5.0, "string_null": "not_null"} ).show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        null|        5.0|
|       null|       World|       null|
|       null|        null|       null|
+-----------+------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        null|        5.0|
|   not_null|       World|        5.0|
|   not_null|        null|        5.0|
+-----------+------------+-----------+



In [108]:
myDf.show()
myDf.na.drop("all", subset=("string_null", "number_null")).show()
myDf.na.drop("all").show()
myDf.na.drop("any", subset=("string_null", "number_null")).show()
myDf.na.drop("any", subset=("string_null", "string2_null", "number_null")).show()
myDf.na.drop("any").show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        null|        5.0|
|       null|       World|       null|
|       null|        null|       null|
+-----------+------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        null|        5.0|
+-----------+------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|       World|        5.0|
|      Hello|        n

### 7.4 조건에 따라 다른 값으로 대체 (na.replace)

In [36]:
""" 조건에 따라 다른 값으로 대체 """
myDf.na.replace([""], ["Hello"], "string_null").show() # null을 지정하는 방법은?

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+



### <font color=blue>8. [중급]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 고객아이디(CustomerID) 컬럼에 대해서만 널 검사를 하되 널값이 있다면 해당 로우는 제외해 주세요
#### 2. 국가(Country) 값은 아래의 규칙으로 변경해 주세요
* "United Kingdom" -> "UK", "France" -> "FR", "Germany" -> "DE", "Netherlands" -> "NL", "Australia" -> "AT", "Norway" -> "NO", "EIRE" -> "IE"

#### 3. 국가(Country) 별 빈도수를 출력해 주세요
* 국가의 빈도수 역순으로 정렬해 주세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 주세요

<details><summary>[실습8] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df8 = spark.read.csv(f"{work_data}/retail-data/by-day/2010-12-01.csv", inferSchema=True, header=True)
keys = ["United Kingdom", "France", "Germany", "Netherlands", "Australia", "Norway", "EIRE"]
values = ["UK", "FR", "DE", "NL", "AT", "NO", "IE"]
answer = (
    df8.na.drop("any", subset=("CustomerID"))
    .na.replace(keys, values, "Country")
    .groupBy("Country")
    .count().orderBy(desc("count"))
).show()
```

</details>


In [119]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


+-------+-----+
|Country|count|
+-------+-----+
|     UK| 1809|
|     NO|   73|
|     DE|   29|
|     IE|   21|
|     FR|   20|
|     AT|   14|
|     NL|    2|
+-------+-----+



## 8. 복합 데이터 다루기
> 구조체, 배열, 맵 등을 스파크에서 다루기

### 8.1 구조체
* DataFrame 내부의 DataFrame
  - 다수의 컬럼을 괄호로 묶어 생성 가능
  - 문법에 점(.)을 사용하거나 getField 메서드를 사용
  - (*) 문자로 모든 값을 조회할 수 있음

In [37]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.show(5, False)
complexDF.printSchema()

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365} |
|{WHITE METAL LANTERN, 536365}                |
|{CREAM CUPID HEARTS COAT HANGER, 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE, 536365}|
|{RED WOOLLY HOTTIE WHITE HEART., 536365}     |
+---------------------------------------------+
only showing top 5 rows

root
 |-- complex: struct (nullable = false)
 |    |-- Description: string (nullable = true)
 |    |-- InvoiceNo: string (nullable = true)



In [38]:
complexDF.select("complex.Description", "complex.InvoiceNo") # 모두 동일
complexDF.select(col("complex").getField("Description"), col("complex").getField("InvoiceNo"))
complexDF.select("complex.*")
complexDF.select(col("complex.*"))
complexDF.selectExpr("complex.*").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



### 8.2 배열
> 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환

#### 컬럼을 구분자로 분리하여 배열로 변환 (split)

In [39]:
""" 컬럼을 배열로 변환 """
from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [40]:
""" 배열값의 조회 """
df.select(split(col("Description"), " ").alias("array_col"))\
    .selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



#### 배열의 길이 (size)

In [41]:
""" size 함수 """
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



#### 배열에 특정 값이 존재하는지 확인 (array_contains)

In [42]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



#### 컬럼의 배열값에 포함된 모든 값을 로우로 변환 (explode)

In [43]:
from pyspark.sql.functions import split, explode
exploded = df \
    .withColumn("splitted", split(col("Description"), " ")) \
    .withColumn("exploded", explode(col("splitted")))
exploded.printSchema()

ef = exploded.select("Description", "InvoiceNo", "exploded") # 모든 단어가 하나의 로우로 전환됨
print(df.select("Description").count())
print(ef.select("exploded").count()) # 로우 수가 다름

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- splitted: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- exploded: string (nullable = true)

3108
14414


In [44]:
exploded.select("Description", "exploded").count() # 큰 쪽으로 카운드

14414

In [45]:
exploded.select("Description", "exploded").take(10) # Description 컬럼이 Group이 되어 중복됨

[Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='WHITE'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HANGING'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HEART'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='T-LIGHT'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HOLDER'),
 Row(Description='WHITE METAL LANTERN', exploded='WHITE'),
 Row(Description='WHITE METAL LANTERN', exploded='METAL'),
 Row(Description='WHITE METAL LANTERN', exploded='LANTERN'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CREAM'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CUPID')]

### 8.3 맵
+ map 함수와 컬럼의 키0값 쌍을 이용해 생성
+ 적합한 키를 사용해 데이터를 조회할 수 있으며, 해당키가 없다면 null값을 반환

In [46]:
""" 맵 생성 """
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(20, False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365} |
|{WHITE METAL LANTERN -> 536365}                |
|{CREAM CUPID HEARTS COAT HANGER -> 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE -> 536365}|
|{RED WOOLLY HOTTIE WHITE HEART. -> 536365}     |
|{SET 7 BABUSHKA NESTING BOXES -> 536365}       |
|{GLASS STAR FROSTED T-LIGHT HOLDER -> 536365}  |
|{HAND WARMER UNION JACK -> 536366}             |
|{HAND WARMER RED POLKA DOT -> 536366}          |
|{ASSORTED COLOUR BIRD ORNAMENT -> 536367}      |
|{POPPY'S PLAYHOUSE BEDROOM  -> 536367}         |
|{POPPY'S PLAYHOUSE KITCHEN -> 536367}          |
|{FELTCRAFT PRINCESS CHARLOTTE DOLL -> 536367}  |
|{IVORY KNITTED MUG COSY  -> 536367}            |
|{BOX OF 6 ASSORTED COLOUR TEASPOONS -> 536367} |
|{BOX OF VINTAGE JIGSAW BLOCKS  -> 536367}      |
|{BOX OF VINTAGE ALPHABET BLOCKS -> 536367}     |


In [47]:
""" 맵의 데이터 조회 """
mapped = df \
    .select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))
mapped.printSchema()
mapped.selectExpr("complex_map['WHITE METAL LANTERN']").where("complex_map['WHITE METAL LANTERN'] is not null").show()

root
 |-- complex_map: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                          536365|
|                          536373|
|                          536375|
|                          536396|
|                          536406|
|                          536544|
+--------------------------------+



In [48]:
""" 맵의 분해 """
exploded = df \
    .select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")) \
    .selectExpr("explode(complex_map)")
exploded.printSchema()
exploded.show(5)

root
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
|CREAM CUPID HEART...|536365|
|KNITTED UNION FLA...|536365|
|RED WOOLLY HOTTIE...|536365|
+--------------------+------+
only showing top 5 rows



### 9. JSON 다루기

In [49]:
""" Json 컬럼 생성 """
jsonDF = spark.range(1).selectExpr(
    """
    '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString
    """
)

In [50]:
""" 인라인 쿼리로 JSON 조회하기 """
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "jsonString.myJSONKey.myJSONValue[1]").alias("column"),
    json_tuple(col("jsonString"), "myJSONKey")
).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|  null|{"myJSONValue":[1...|
+------+--------------------+



In [51]:
""" StructType을 Json 문자열로 변경 """
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct") \
    .select(to_json(col("myStruct"))) \
    .take(3)

[Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}'),
 Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}'),
 Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}')]

In [52]:
""" Json 문자열을 객체로 변환 """
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("Description", StringType(), True)
])

df.selectExpr("(InvoiceNo, Description) as myStruct") \
    .select(to_json(col("myStruct")).alias("newJSON")) \
    .select(from_json(col("newJSON"), parseSchema), col("newJSON")) \
    .show(2) # 키를 컬럼명으로 값을 로우로 변경

+--------------------+--------------------+
|  from_json(newJSON)|             newJSON|
+--------------------+--------------------+
|{536365, WHITE HA...|{"InvoiceNo":"536...|
|{536365, WHITE ME...|{"InvoiceNo":"536...|
+--------------------+--------------------+
only showing top 2 rows



### <font color=green>9. [기본]</font> f"{work_data}/retail-data/by-day/2010-12-01.csv" 에 저장된 CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요
#### 3. 고객구분자(CustomerID)와 설명(Description) 컬럼이 널값인 데이터프레임을 추출하여 출력하세요
#### 4. 고객구분자(CustomerID)가 null 인 경우는 0.0 으로 치환하고
#### 5. 설명(Description)가 null 인 경우는 "NOT MENTIONED" 값으로 저장될 수 있도록 만들어주세요
#### 6. 최종 스키마와 데이터를 출력해 주세요

<details><summary>[실습9] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df5 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/by-day/2010-12-01.csv")
).where(expr("Description is null or CustomerID is null"))
df5.printSchema()
df5.show(10)
desc_custid_fill = {"Description":"NOT MENTIONED", "CustomerID":0.0}
answer = df5.na.fill(desc_custid_fill)
answer.printSchema()
display(answer)
```

</details>


In [55]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536414|    22139|                null|      56|2010-12-01 11:52:00|      0.0|      null|United Kingdom|
|   536544|    21773|DECORATIVE ROSE B...|       1|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21774|DECORATIVE CATS B...|       2|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536414,22139,NOT MENTIONED,56,2010-12-01 11:52:00,0.0,0.0,United Kingdom
536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,0.0,United Kingdom
536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,0.0,United Kingdom
536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,0.0,United Kingdom
536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,0.0,United Kingdom
536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,0.0,United Kingdom
536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,2010-12-01 14:32:00,2.51,0.0,United Kingdom
536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,2010-12-01 14:32:00,0.43,0.0,United Kingdom
536544,21802,CHRISTMAS TREE HEART DECORATION,9,2010-12-01 14:32:00,0.43,0.0,United Kingdom
536544,21803,CHRISTMAS TREE STAR DECORATION,11,2010-12-01 14:32:00,0.43,0.0,United Kingdom


## 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
#### 4. [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
#### 5. [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)